# Project milestone 4
# Detection of housing-health relationship


The aim of this project is to estimate the relation between housing quality and health status of a person. The project is an observational study based on a survey made by Mexican National Institute of Statistics and Geography. The health variables reported for each person will be used to define a single health variable which will be a score for each person's health status. We aim to use machine learning methods to do the classification and we will use regression models to predict this health score from the housing variables. Matching will be used to weed out the possible covariates. The motivation is to estimate the most important parameters of housing quality so that we can propose most cost-effective solutions that would increase the quality of health. The original paper is based on the analysis of influence of concrete floors on health quality, while here we would investigate some other parameters such as material used for building and whether there is a toilet or not in the household.

# Step 1: Import data

In [48]:
import pandas as pd
import numpy as np

### Import data ###
#data_household = pd.read_csv('data_translated/household.csv')
data_house = pd.read_csv('data_translated/house.csv',low_memory=False)
data_person = pd.read_csv('data_translated/person.csv',low_memory=False)

In [49]:
#data_person_all = data_person.merge(data_house, left_on='House identifier', right_on='House identifier')
data_person_all = data_person.merge(data_house, left_on=['House_identifier'], right_on=['House_identifier'])

# Step 2: Exploratory Data Analysis

In [50]:
# our selected health variables
health_var = list(data_person_all.columns[35:57]) + [data_person_all.columns[26]]
data_person_all[health_var].describe()

,Difficulty_seeing,Wear_a_hearing_aid,Difficulty_hearing,Difficulty_hearing_without_noise,Difficulty_hearing_with_noise,Dificulty_to_walk,Use_a_walking_device,Walking_apparatus,Difficulty_walking_100_m,Difficulty_walking_500_m,...,Medication_for_nervousness,Intensity_of_nervousness,Frequency_of_depression,Antidepressant_medications,Intensity_of_depression,Frequency_of_pain,Pain_intensity,Fatigue_frequency,Tired_time,Limiting_physical_or_mental_activity
count,208140,208140,208140,208140,208140,208140,208140,208140,208140,208140,...,208140,208140,208140,208140,208140,208140,208140,208140,208140,208140
unique,6,3,6,6,6,6,3,9,6,6,...,4,5,7,4,5,6,5,6,5,2
top,1,2,1,1,1,1,2,,1,1,...,2,,5,2,,1,,1,,
freq,163575,197136,186322,190264,177606,175302,191991,201868,184590,177483,...,178203,117286,126698,180272,152071,132028,157153,129517,154635,205250


In [70]:
# replacing " " values with proper None
data_person_all = data_person_all.replace(' ', np.nan)

['House_identifier',
 'Household_identifier',
 'Identifier_of_the_person',
 'Age',
 'Birthday',
 'Birth_month',
 'Sex',
 'Relationship',
 'School_attendance',
 'School_type',
 'School_shift',
 'Scholarship_or_educational_credit',
 'Awards_school_scholarship',
 'Scholarship_form',
 'Reason_for_non-attendance',
 'Level_of_instruction',
 'Level_of_instruction.1',
 'Home',
 'Literacy',
 'Marital_status',
 'Worked_last_week',
 'Reason_for_absence_from_work',
 'Job_search_activity_',
 'Retired_or_retired_',
 'Housework_activity_',
 'Student_activity',
 'Limiting_physical_or_mental_activity',
 'Activity_from_a_different_situation',
 'Job_numbers',
 'He_was_subordinate',
 'Work_on_your_own_',
 'Had_staff',
 'Payment_method',
 'Had_a_written_contract',
 'Wear_glasses',
 'Difficulty_seeing',
 'Wear_a_hearing_aid',
 'Difficulty_hearing',
 'Difficulty_hearing_without_noise',
 'Difficulty_hearing_with_noise',
 'Dificulty_to_walk',
 'Use_a_walking_device',
 'Walking_apparatus',
 'Difficulty_walking_

In [75]:
# drop columns with more than 30% of NaN values
nulli = []
exc = []
thr = int(len(data_person_all)*0.3)
for i in range(len(list(data_person_all.columns))):
    nulli.append(data_person_all[list(data_person_all.columns)[i]].isnull().values.sum())
    if (nulli[i] > thr):
        exc.append(i)
exc
# there are a lot of NaNs

[9,
 10,
 11,
 12,
 13,
 14,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 42,
 49,
 52,
 54,
 56,
 57,
 59,
 60,
 83,
 84,
 85,
 86,
 90,
 125,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152]

# Step X: Matching